In [0]:
%sql
USE CATALOG pricing_analytics;
SELECT current_catalog(), current_schema();



In [0]:
processName = dbutils.widgets.get("prm_processName")

nextSourceFileDateSql = f"""select max(PROCESS_FILE_TABLE_DATE)+1 as NEXT_SOURCE_FILE_DATE from processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS where PROCESS_NAME='{processName}' and PROCESS_STATUS='Completed'"""

In [0]:
nextSourceFileDateDf = spark.sql(nextSourceFileDateSql)

In [0]:
print(nextSourceFileDateDf.select("NEXT_SOURCE_FILE_DATE").collect()[0]["NEXT_SOURCE_FILE_DATE"])

In [0]:
from datetime import datetime
dbutils.widgets.text("prm_dailyPricingSourceFileDate", "")
print(datetime.strptime(str(nextSourceFileDateDf.select("NEXT_SOURCE_FILE_DATE").collect()[0]["NEXT_SOURCE_FILE_DATE"]), "%Y-%m-%d").strftime("%d%m%Y"))

In [0]:
sourceFileURL = 'https://retailpricing.blob.core.windows.net/labs/lab1/PW_MW_DR_01012023.csv'
bronzelayerCSVFilePath = 'abfss://working-labs@adlsudadalakehousedev.dfs.core.windows.net/bronze/daily-pricing/csv'


In [0]:
dailyPricingSourceBaseURL = "https://retailpricing.blob.core.windows.net/"
dailyPricingSourceFileDate = datetime.strptime(dbutils.widgets.get("prm_dailyPricingSourceFileDate"), "%Y-%m-%d").strftime("%d%m%Y")
dailyPricingSourceFile = f"PW_MW_DR_{dailyPricingSourceFileDate}.csv"
dailyPricingSourceFolder = "daily-pricing/"

dailyPricingSinkLayerName = "bronze"
dailyPricingSinkStorageAccountName = "adlsudadalakehousedev"
dailyPricingSinkFolderName = "daily-pricing"


In [0]:
dailyPricingSourceFile

In [0]:
import pandas as pds

In [0]:
dailyPricingSourceURL = dailyPricingSourceBaseURL + dailyPricingSourceFolder + dailyPricingSourceFile 
print(dailyPricingSourceURL)

In [0]:
dailyPricingPandasDF = pds.read_csv(dailyPricingSourceURL)

In [0]:
 dailyPricingPandasDF

In [0]:
dailyPricingSparkDF = spark.createDataFrame(dailyPricingPandasDF)

In [0]:
dailyPricinSinkFolderPath = f"abfss://{dailyPricingSinkLayerName}@{dailyPricingSinkStorageAccountName}.dfs.core.windows.net/{dailyPricingSinkFolderName}"

In [0]:
print(dailyPricinSinkFolderPath)

In [0]:
(
    dailyPricingSparkDF.
    write.
    mode("overwrite").
    option("header","true").
    csv(dailyPricinSinkFolderPath)
)

In [0]:
dbutils.fs.ls(dailyPricinSinkFolderPath)

In [0]:
%sql
USE CATALOG pricing_analytics;

CREATE SCHEMA IF NOT EXISTS processrunlogs;

CREATE TABLE IF NOT EXISTS processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS(
    PROCESS_NAME STRING, 
    PROCESS_FILE_TABLE_DATE DATE,
    PROCESS_STATUS STRING
)

In [0]:
processName = 'dailyPricingSourceIngest'
processFileDate = dbutils.widgets.get("prm_dailyPricingSourceFileDate")
processStatus = "Completed"

processInsertSql = f"INSERT INTO processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS VALUES ('{processName}','{processFileDate}','{processStatus}')"


In [0]:
spark.sql(processInsertSql)

In [0]:
%sql
select * from  processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS 

In [0]:
%sql
select max(PROCESS_FILE_TABLE_DATE)+1 from processrunlogs.deltalakehouse_process_runs where PROCESS_NAME="dailyPricingSourceIngest" and PROCESS_STATUS="Completed"